In [160]:
#import necessary libraries
import pandas as pd
from pandas.io.json import json_normalize

%matplotlib inline

In [161]:
def read_station(hourstamp):
    bike_data=pd.read_json("../Data_raw/station"+str(hourstamp)+".json", orient='records')
    station=json_normalize(bike_data["stations"])

    # Clean column names
    station=station[["id", "ba","da"]]
    station["Empty_ratio"] = station.ba / (station.ba + station.da)
    
    station=station.drop(["ba", "da"], axis=1)
    station.columns = ["ID", "Empty_ratio"]
    return station

In [162]:
hourly_empty_station = station_zip[["ID", "Latitude", "Longitude", "Station"]]
for i in range(24):
    hourly_empty_station = pd.merge(hourly_empty_station, read_station(i), how="left", left_on="ID", right_on="ID")                                       

In [163]:
hourly_empty_station.columns =["ID", "Latitude", "Longitude", "Station"]+list(range(24))

In [164]:
hourly_empty_station["mean"]=hourly_empty_station.iloc[:, 4:].mean(axis=1)
hourly_empty_station["median"]=hourly_empty_station.iloc[:, 4:].median(axis=1)

In [165]:
hourly_empty_station.head(3)

,ID,Latitude,Longitude,Station,0,1,2,3,4,5,...,16,17,18,19,20,21,22,23,mean,median
0,3,42.340115,-71.100619,Colleges of the Fenway - Fenway at Avenue Loui...,0.800000,0.733333,0.733333,0.733333,0.733333,0.733333,...,0.133333,0.200000,0.000000,0.200000,0.200000,0.266667,0.666667,0.600000,0.550000,0.666667
1,4,42.345392,-71.069616,Tremont St at E Berkeley St,1.000000,0.705882,0.705882,0.764706,0.764706,0.764706,...,0.470588,0.941176,0.823529,0.764706,0.705882,0.823529,0.823529,0.882353,0.764706,0.764706
2,5,42.341814,-71.090179,Northeastern University - North Parking Lot,0.083333,0.083333,0.083333,0.083333,0.083333,0.916667,...,0.000000,0.066667,0.200000,0.266667,0.133333,0.083333,0.083333,0.083333,0.197917,0.083333


In [167]:
hourly_empty_station.to_csv("../Data_processed/station_hours.csv")